In [2]:
import sys
import pathlib
import torch

import numpy as np

# Local
cwd = pathlib.Path().resolve()
src = cwd.parent
root = src.parent
sys.path.append(str(src))

from models.unet import UNet
from models.unet_source import UNet_source
from models.unet_mask import UNet_mask

from utils.watertopo import WaterTopo
from utils.utils import recursive_pred, mse_per_timestep

%matplotlib inline

### Test 1
Load the simulations of test 1, create inputs and targets

In [33]:
grid_size = 64
channels = 2

sims = WaterTopo.load_simulations(str(root)+"/data/normalized_data/test1", sim_amount=20, number_grids=grid_size, use_augmented_data=True)

dur = len(sims[0].wd)

X   = np.zeros((len(sims), 1, 2, grid_size, grid_size))
Y0  = np.zeros((len(sims), dur, 1, grid_size, grid_size))
Y5  = Y0[:,::5+1,:,:,:]
Y10 = Y0[:,::10+1,:,:,:]

for i,sim in enumerate(sims):
    X[i,:,0,:,:] = sim.topography
    X[i,:,1,:,:] = sim.wd[0]

    Y0[i,:,0,:,:]  = sim.wd
    Y5[i,:,0,:,:]  = sim.implement_skips(5).wd
    Y10[i,:,0,:,:] = sim.implement_skips(10).wd

print("X: ", X.shape)
print("Y0: ", Y0.shape)
print("Y5: ", Y5.shape)
print("Y10: ", Y10.shape)


X:  (20, 1, 2, 64, 64)
Y0:  (20, 97, 1, 64, 64)
Y5:  (20, 17, 1, 64, 64)
Y10:  (20, 9, 1, 64, 64)


Multistep-models

In [58]:
# Load the model
model = UNet(2, [32, 64, 128], 96)
model_type = "unet"
model_name = "unet_32_64_128_orig_data80_multistep_skip0"
Y = Y0
skips = 0

model.load_state_dict(torch.load(str(src)+"/results/trained_models/" + model_type + "/" + model_name, 
                                 map_location="cpu"))

mse = np.zeros((len(sims), Y.shape[1]))

for i, sim in enumerate(sims):
    inputs = torch.tensor(X[i], dtype=torch.float32)
    outputs = model(inputs).squeeze()

    outputs = torch.cat((torch.tensor(sim.wd[0], dtype=torch.float32).unsqueeze(0), outputs), dim=0)
    
    mse[i,:] = mse_per_timestep(Y[i].squeeze(), outputs)

mse = np.mean(mse, axis=0)
t = np.arange(0, 97, skips+1)
mse = np.vstack([t, mse])

np.savetxt(str(src)+"/results/error_accumulation/"+model_name, mse)

Auto-regressive models

In [77]:
# Load the model
model = UNet(2, [32, 64], 1)
model_type = "unet"
model_name = "unet_32_64_orig_data80_skip10"
Y = Y10
skips = 10

model.load_state_dict(torch.load(str(src)+"/results/trained_models/" + model_type + "/" + model_name, 
                                 map_location="cpu"))
model.eval()

mse = np.zeros((len(sims), Y.shape[1]))

for i, sim in enumerate(sims):
    inputs = torch.tensor(X[i], dtype=torch.float32).squeeze()

    with torch.no_grad():
        outputs = recursive_pred(model, inputs, Y.shape[1]-1, include_first_timestep=True)
    
    mse[i,:] = mse_per_timestep(Y[i].squeeze(), outputs)

mse = np.mean(mse, axis=0)
t = np.arange(0, 97, skips+1)
mse = np.vstack([t, mse])

np.savetxt(str(src)+"/results/error_accumulation/"+model_name, mse)

In [80]:
# Load the model
model = UNet_mask(2, [32, 64], 1)
model_type = "unet_mask"
model_name = "unet_32_64_orig_data80_skip5_softmask"
Y = Y5
skips = 5

model.load_state_dict(torch.load(str(src)+"/results/trained_models/" + model_type + "/" + model_name, 
                                 map_location="cpu"))
model.eval()

mse = np.zeros((len(sims), Y.shape[1]))

for i, sim in enumerate(sims):
    inputs = torch.tensor(X[i], dtype=torch.float32).squeeze()

    with torch.no_grad():
        outputs = recursive_pred(model, inputs, Y.shape[1]-1, include_first_timestep=True)
    
    mse[i,:] = mse_per_timestep(Y[i].squeeze(), outputs)

mse = np.mean(mse, axis=0)
t = np.arange(0, 97, skips+1)
mse = np.vstack([t, mse])

np.savetxt(str(src)+"/results/error_accumulation/"+model_name, mse)